In [11]:
import numpy as np
import pandas as pd
import os
import json

cwd = os.getcwd()

results_path = cwd[:-28] + "Results/RandomN_test/all_scores_range_0_137_trail_{}.npz"
results_path_sbert = cwd[:-28] + "Results/RandomN_test/all_sbert_scores_0_137_trail_{}.npz"

results_intent_path = cwd[:-28] + "Results/RandomN_test/all_scores_range_by_intent_0_137_trail_{}.txt"

intents = ["What drives the economy in this state?",
           "What are the available modes of transport in this state?",
           "How is the government structured in this state?",
           "What about this state's arts and culture attracts you the most?",
           "The main reasons why you would like living in this state"]

sample_size = 2

model = "RandomN"

In [12]:

total_data = None
total_data_sbert = []
total_data_intent = {}

for intent in intents:
    total_data_intent[intent] = []

# get all of the "normal" data
for i in range(sample_size):
    temp = np.load(results_path.format(i))
    sbert_temp = np.load(results_path_sbert.format(i))
    data = temp['scores']
    data_sbert = sbert_temp['scores']
    
    if total_data is None:
        # we need to take the mean of axis-one first because each "user-sample" contains a set of predicted summaries. So we first avg those sets, before average across all the data/trials
        total_data = data.mean(axis=1)
        total_data_sbert = data_sbert.mean(axis=1)
    else:
        total_data = np.concatenate((total_data, data.mean(axis=1)), axis=0)
        total_data_sbert = np.concatenate((total_data_sbert, data_sbert.mean(axis=1)), axis=0)

# get all of the "by-intent" data
for i in range(sample_size):
    res = None
    with open(results_intent_path.format(i)) as f:
        res = json.load(f)

    for intent in intents:
        total_data_intent[intent].extend(np.mean(res[intent], axis=1).tolist())

# now calculate the relavent stats
total_mean = np.mean(total_data, axis=0)
total_stddev = np.std(total_data, axis=0)

total_mean_sbert = np.mean(total_data_sbert, axis=0)
total_stddev_sbert = np.std(total_data_sbert, axis=0)

mean_intent = {}
stddev_intent = {}

for intent in intents:
    mean = np.mean(total_data_intent[intent], axis=0)
    stddev = np.std(total_data_intent[intent], axis=0)
    mean_intent[intent] = mean
    stddev_intent[intent] = stddev

metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for i, (row_m, row_s) in enumerate(zip(total_mean, total_stddev)):
    row_str = model + " & "
    for col_m, col_s in zip(row_m, row_s):
        row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
    
    row_str = row_str[:-3]
    row_str += " \\\\"
    print(metrics[i])
    print(row_str)
print("SBERT Cosime Sim. Score: ")
print(model + " & " + str(np.round(total_mean_sbert, 4)) + " $\\pm$ " + str(np.round(total_stddev_sbert, 4)) + " \\\\")

print("\n\n\n")

Rouge-1
RandomN & 0.3395 $\pm$ 0.0624 & 0.3258 $\pm$ 0.0598 & 0.3195 $\pm$ 0.0436 & 0.1997 $\pm$ 0.0272 \\
Rouge-2
RandomN & 0.0859 $\pm$ 0.0383 & 0.0823 $\pm$ 0.0346 & 0.0807 $\pm$ 0.0332 & 0.0505 $\pm$ 0.0208 \\
Rouge-L
RandomN & 0.1776 $\pm$ 0.0359 & 0.1685 $\pm$ 0.0313 & 0.1658 $\pm$ 0.0229 & 0.1036 $\pm$ 0.0143 \\
SBERT Cosime Sim. Score: 
RandomN & 0.4661 $\pm$ 0.0719 \\






In [15]:
metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for intent in intents:
    print("\nResults for intent {}\n".format(intent))
    for i, (row_m, row_s) in enumerate(zip(mean_intent[intent], stddev_intent[intent])):
        row_str = "\t"+model + " & "
        for col_m, col_s in zip(row_m, row_s):
            row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
        
        row_str = row_str[:-3]
        row_str += " \\\\"
        print("\t"+metrics[i])
        print(row_str)


Results for intent What drives the economy in this state?

	Rouge-1
	LeadN & 0.3331 $\pm$ 0.0649 & 0.3226 $\pm$ 0.0497 & 0.3163 $\pm$ 0.0444 & 0.1977 $\pm$ 0.0278 \\
	Rouge-2
	LeadN & 0.0845 $\pm$ 0.0407 & 0.0803 $\pm$ 0.0316 & 0.0795 $\pm$ 0.0341 & 0.0497 $\pm$ 0.0213 \\
	Rouge-L
	LeadN & 0.1732 $\pm$ 0.0338 & 0.1666 $\pm$ 0.0264 & 0.1637 $\pm$ 0.0218 & 0.1023 $\pm$ 0.0137 \\

Results for intent What are the available modes of transport in this state?

	Rouge-1
	LeadN & 0.3164 $\pm$ 0.0556 & 0.3083 $\pm$ 0.0628 & 0.2983 $\pm$ 0.0381 & 0.1865 $\pm$ 0.0238 \\
	Rouge-2
	LeadN & 0.0764 $\pm$ 0.0343 & 0.0747 $\pm$ 0.0329 & 0.0721 $\pm$ 0.0293 & 0.0451 $\pm$ 0.0183 \\
	Rouge-L
	LeadN & 0.1671 $\pm$ 0.034 & 0.1599 $\pm$ 0.0317 & 0.1557 $\pm$ 0.0199 & 0.0973 $\pm$ 0.0125 \\

Results for intent How is the government structured in this state?

	Rouge-1
	LeadN & 0.3785 $\pm$ 0.0591 & 0.3109 $\pm$ 0.0592 & 0.3275 $\pm$ 0.0419 & 0.2047 $\pm$ 0.0262 \\
	Rouge-2
	LeadN & 0.0975 $\pm$ 0.0385 & 0.082

In [16]:
results_path = cwd[:-28] + "Results/LeadN_test/all_scores_range_0_137_trail_{}.npz"
results_path_sbert = cwd[:-28] + "Results/LeadN_test/all_sbert_scores_0_137_trail_{}.npz"

results_intent_path = cwd[:-28] + "Results/LeadN_test/all_scores_range_by_intent_0_137_trail_{}.txt"

model = "LeadN"

In [17]:

total_data = None
total_data_sbert = []
total_data_intent = {}

for intent in intents:
    total_data_intent[intent] = []

# get all of the "normal" data
for i in range(sample_size):
    temp = np.load(results_path.format(i))
    sbert_temp = np.load(results_path_sbert.format(i))
    data = temp['scores']
    data_sbert = sbert_temp['scores']
    
    if total_data is None:
        # we need to take the mean of axis-one first because each "user-sample" contains a set of predicted summaries. So we first avg those sets, before average across all the data/trials
        total_data = data.mean(axis=1)
        total_data_sbert = data_sbert.mean(axis=1)
    else:
        total_data = np.concatenate((total_data, data.mean(axis=1)), axis=0)
        total_data_sbert = np.concatenate((total_data_sbert, data_sbert.mean(axis=1)), axis=0)

# get all of the "by-intent" data
for i in range(sample_size):
    res = None
    with open(results_intent_path.format(i)) as f:
        res = json.load(f)

    for intent in intents:
        total_data_intent[intent].extend(np.mean(res[intent], axis=1).tolist())

# now calculate the relavent stats
total_mean = np.mean(total_data, axis=0)
total_stddev = np.std(total_data, axis=0)

total_mean_sbert = np.mean(total_data_sbert, axis=0)
total_stddev_sbert = np.std(total_data_sbert, axis=0)

mean_intent = {}
stddev_intent = {}

for intent in intents:
    mean = np.mean(total_data_intent[intent], axis=0)
    stddev = np.std(total_data_intent[intent], axis=0)
    mean_intent[intent] = mean
    stddev_intent[intent] = stddev

metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for i, (row_m, row_s) in enumerate(zip(total_mean, total_stddev)):
    row_str = model + " & "
    for col_m, col_s in zip(row_m, row_s):
        row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
    
    row_str = row_str[:-3]
    row_str += " \\\\"
    print(metrics[i])
    print(row_str)
print("SBERT Cosime Sim. Score: ")
print(model + " & " + str(np.round(total_mean_sbert, 4)) + " $\\pm$ " + str(np.round(total_stddev_sbert, 4)) + " \\\\")

print("\n\n\n")

Rouge-1
LeadN & 0.3607 $\pm$ 0.0595 & 0.3395 $\pm$ 0.0747 & 0.3362 $\pm$ 0.0523 & 0.2101 $\pm$ 0.0327 \\
Rouge-2
LeadN & 0.0995 $\pm$ 0.0548 & 0.0942 $\pm$ 0.0524 & 0.0931 $\pm$ 0.0511 & 0.0582 $\pm$ 0.0319 \\
Rouge-L
LeadN & 0.1987 $\pm$ 0.0477 & 0.1838 $\pm$ 0.0475 & 0.1833 $\pm$ 0.0403 & 0.1145 $\pm$ 0.0252 \\
SBERT Cosime Sim. Score: 
LeadN & 0.5388 $\pm$ 0.0782 \\






In [18]:
metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for intent in intents:
    print("\nResults for intent {}\n".format(intent))
    for i, (row_m, row_s) in enumerate(zip(mean_intent[intent], stddev_intent[intent])):
        row_str = "\t"+model + " & "
        for col_m, col_s in zip(row_m, row_s):
            row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
        
        row_str = row_str[:-3]
        row_str += " \\\\"
        print("\t"+metrics[i])
        print(row_str)


Results for intent What drives the economy in this state?

	Rouge-1
	LeadN & 0.3632 $\pm$ 0.0636 & 0.3337 $\pm$ 0.0603 & 0.3348 $\pm$ 0.046 & 0.2092 $\pm$ 0.0288 \\
	Rouge-2
	LeadN & 0.1117 $\pm$ 0.0554 & 0.1005 $\pm$ 0.0428 & 0.1016 $\pm$ 0.0452 & 0.0635 $\pm$ 0.0283 \\
	Rouge-L
	LeadN & 0.2002 $\pm$ 0.0483 & 0.1809 $\pm$ 0.0396 & 0.1828 $\pm$ 0.036 & 0.1142 $\pm$ 0.0225 \\

Results for intent What are the available modes of transport in this state?

	Rouge-1
	LeadN & 0.3355 $\pm$ 0.0469 & 0.3262 $\pm$ 0.075 & 0.3161 $\pm$ 0.04 & 0.1976 $\pm$ 0.025 \\
	Rouge-2
	LeadN & 0.0768 $\pm$ 0.0191 & 0.0765 $\pm$ 0.0265 & 0.0734 $\pm$ 0.0197 & 0.0459 $\pm$ 0.0123 \\
	Rouge-L
	LeadN & 0.179 $\pm$ 0.0246 & 0.1702 $\pm$ 0.036 & 0.1662 $\pm$ 0.0144 & 0.1039 $\pm$ 0.009 \\

Results for intent How is the government structured in this state?

	Rouge-1
	LeadN & 0.3763 $\pm$ 0.0371 & 0.3063 $\pm$ 0.0608 & 0.323 $\pm$ 0.0313 & 0.2019 $\pm$ 0.0196 \\
	Rouge-2
	LeadN & 0.0808 $\pm$ 0.0164 & 0.0657 $\pm$ 0